In [10]:

import cv2 as cv 
import dlib  
import numpy as np  
import tensorflow as tf 
import matplotlib.pyplot as plt  
from scipy.spatial import distance as dist 
import face_recognition as fr
#----------------------------------------------ZMIENNE---------------------------------
EYE_AR_THRESH, EYE_AR_CONSEC_FRAMES, OPEN_EYES_CONSEC_FRAMES = 0.25, 20, 20  #Próg zamknięcia oczu, liczba klatek przy zamkniętych, liczba klatek przy otwwartych
yawn_treshhold = 0.7
blink_counter, open_counter, count_yawn = 0, 0, 0 #flagi
drowsy_alert = False #Flaga ostrzeżenia

#-------------------------------------------FUNKCJE-------------------------------------
#Funkcja obliczająca Eye Aspect Ratio
ear = lambda eye: (dist.euclidean(eye[1], eye[5]) + dist.euclidean(eye[2], eye[4])) / (2.0 * dist.euclidean(eye[0], eye[3]))
mar = lambda mouth: (dist.euclidean(mouth[2], mouth[10]) + dist.euclidean(mouth[4], mouth[8])) / (2.0 * dist.euclidean(mouth[0], mouth[6]))

# def eye_aspect_ratio(eye):
#     A = dist.euclidean(eye[1], eye[5])  #Odległość między górną i dolną powieką (wewnętrzna część oka)
#     B = dist.euclidean(eye[2], eye[4])  #Odległość między górną i dolną powieką (zewnętrzna część oka)
#     C = dist.euclidean(eye[0], eye[3])  #Odległość między kącikami oka
#     return (A + B) / (2.0 * C)  #Wskaźnik EAR - im niższy, tym bardziej zamknięte oczy

# def mouth_aspect_ratio(mouth):
#     A = dist.euclidean(mouth[2], mouth[10])
#     B = dist.euclidean(mouth[4], mouth[8])
#     C = dist.euclidean(mouth[0], mouth[6])   
#     return (A + B) / (2.0 * C)             

#--------------------------------------------------------------------------------------

#Inicjalizacja detektora twarzy, modelu predykcji, kamery
detector = dlib.get_frontal_face_detector() 
#detector = dlib.cnn_face_detection_model_v1("mmod_human_face_detector.dat") #CNN strasznie się zwiesza
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
cap = cv.VideoCapture(0) 

#Pobranie klatki obrazu z kamery
while True:
    ret, frame = cap.read()
    if not ret:  
        break

    #Wykrywanie twarzy
    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
    #for face in detector(gray): #cnn
        #Pobranie punktów charakterystycznych twarzy (oczy, nos, usta)
        #rect = face.rect #cnn

        landmarks = predictor(gray, face)
        left_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(36, 42)])
        right_eye = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(42, 48)])
        mouth_pts = np.array([(landmarks.part(i).x, landmarks.part(i).y) for i in range(48, 68)])
        cv.rectangle(frame, (face.left(), face.top()), (face.right(), face.bottom()), (0, 0, 255), 2)

        #Obliczenie wskaźnika EAR dla obu oczu, MAR dla ust
        EAR = (ear(left_eye) + ear(right_eye)) / 2.0  
        MAR = mar(mouth_pts)
        # left_EAR = ear(left_eye)
        # right_EAR = ear(right_eye)
        # EAR = (left_EAR + right_EAR) / 2.0  # Średnia wartość EAR dla obu oczu
        

        if EAR<=EYE_AR_THRESH:
            blink_counter+=1
            open_counter=0
        else:
            blink_counter=0
            open_counter+=1

        #Ustawienie alertów
        ear_alert = blink_counter>=EYE_AR_CONSEC_FRAMES
        mar_alert = MAR >= yawn_treshhold
        drowsy_alert = ear_alert or mar_alert

        if open_counter >= OPEN_EYES_CONSEC_FRAMES and not mar_alert:
            drowsy_alert = False

    #Status senności + informacja
    cv.putText(frame, f"Sleepy: {drowsy_alert}", (10, 30),
    cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
    cv.putText(frame, f"EAR: {EAR}", (10, 60),
    cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv.putText(frame, f"MAR: {MAR}", (10, 90),
    cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv.putText(frame, f"Mrugniecie: {blink_counter}", (10, 120),
    cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    cv.putText(frame, f"Otwarte oczy: {open_counter}", (10, 150),
    cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)


    cv.imshow("Kamera", frame)
    if cv.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv.destroyAllWindows()


In [19]:
!git add .
